In [1]:
import getpass
import subprocess
from urllib.parse import quote

username = input("GitHub username: ")
token = getpass.getpass("GitHub token: ")
repo_url = "https://github.com/ISE-Lab-AI4LIFE/SANNER_2025.git"

auth_url = repo_url.replace("https://", f"https://{quote(username)}:{quote(token)}@")

try:
    subprocess.run(["git", "clone", auth_url], check=True)
    print("✅ Repo cloned successfully!")
except subprocess.CalledProcessError as e:
    print("❌ Clone failed. Check error message below:")
    print(e.stderr)

GitHub username: hieunguyen-cyber
GitHub token: ··········
✅ Repo cloned successfully!


In [2]:
from pathlib import Path
import pandas as pd

DATA_DIR = Path("/content/SANNER_2025/data")  # <-- sửa ở đây
HOTFLIP_FILE = DATA_DIR / "hotflip_result" / "merged_hotflip_results.csv"
POOL_FILE = DATA_DIR / "pool.csv"

# --- Đọc dữ liệu ---
hotflip_df = pd.read_csv(HOTFLIP_FILE)
pool_df = pd.read_csv(POOL_FILE)

# --- Lấy danh sách id ---
hotflip_ids = set(hotflip_df["document_id"])
pool_ids = set(pool_df["document_id"])

# --- Phân loại ---
poisoned_doc = hotflip_df.copy()
targeted_doc = pool_df[pool_df["document_id"].isin(hotflip_ids)]
clean_doc = pool_df[~pool_df["document_id"].isin(hotflip_ids)]

# --- In thống kê ---
print(f"Tổng số document trong pool: {len(pool_df)}")
print(f"Số poisoned_doc: {len(poisoned_doc)}")
print(f"Số targeted_doc: {len(targeted_doc)}")
print(f"Số clean_doc: {len(clean_doc)}")

# --- Lưu ra file ---
OUTPUT_DIR = DATA_DIR / "hotflip_result" / "split_docs"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

poisoned_doc.to_csv(OUTPUT_DIR / "poisoned_doc.csv", index=False)
targeted_doc.to_csv(OUTPUT_DIR / "targeted_doc.csv", index=False)
clean_doc.to_csv(OUTPUT_DIR / "clean_doc.csv", index=False)

print(f"✅ Đã lưu 3 file vào: {OUTPUT_DIR}")

Tổng số document trong pool: 5446
Số poisoned_doc: 157
Số targeted_doc: 157
Số clean_doc: 5289
✅ Đã lưu 3 file vào: /content/SANNER_2025/data/hotflip_result/split_docs


In [3]:
import pandas as pd
from pathlib import Path
import numpy as np


DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
SPLIT_DIR = HOTFLIP_DIR / "split_docs"
POISONED_POOL_DIR = HOTFLIP_DIR / "poisoned_pool"
BASE_POOL_DIR = HOTFLIP_DIR / "base_pool"

POISONED_POOL_DIR.mkdir(parents=True, exist_ok=True)
BASE_POOL_DIR.mkdir(parents=True, exist_ok=True)

# --- Đọc 3 file ---
poisoned_doc = pd.read_csv(SPLIT_DIR / "poisoned_doc.csv")
targeted_doc = pd.read_csv(SPLIT_DIR / "targeted_doc.csv")
clean_doc = pd.read_csv(SPLIT_DIR / "clean_doc.csv")

# --- Tạo pool với nhiều ratio ---
ratios = np.array([0.001, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03])

for ratio in ratios:
    # --- Số lượng clean cần chọn ---
    n_poison = len(poisoned_doc)
    n_target = len(targeted_doc)

    n_clean_poisoned_pool = int(n_poison * (1 - ratio) / ratio)
    n_clean_base_pool = int(n_target * (1 - ratio) / ratio)

    # --- Giới hạn nếu clean_doc không đủ ---
    n_clean_poisoned_pool = min(n_clean_poisoned_pool, len(clean_doc))
    n_clean_base_pool = min(n_clean_base_pool, len(clean_doc))

    # --- Sample clean_doc ---
    clean_sample_for_poisoned = clean_doc.sample(
        n=n_clean_poisoned_pool,
        random_state=42,
        replace=False
    )
    clean_sample_for_base = clean_doc.sample(
        n=n_clean_base_pool,
        random_state=42,
        replace=False
    )

    # --- Gắn nhãn chosen ---
    poisoned_part = poisoned_doc[["document_id", "final_poisoned_doc"]].rename(
        columns={"final_poisoned_doc": "document"}
    )
    poisoned_part["chosen"] = 1

    targeted_part = targeted_doc[["document_id", "document"]].copy()
    targeted_part["chosen"] = 1

    clean_part_for_poisoned = clean_sample_for_poisoned[["document_id", "document"]].copy()
    clean_part_for_poisoned["chosen"] = 0

    clean_part_for_base = clean_sample_for_base[["document_id", "document"]].copy()
    clean_part_for_base["chosen"] = 0

    # --- Gộp lại ---
    poisoned_pool = pd.concat([poisoned_part, clean_part_for_poisoned], ignore_index=True)
    base_pool = pd.concat([targeted_part, clean_part_for_base], ignore_index=True)

    # --- Lưu ra CSV ---
    r_str = f"{ratio*100:.1f}".replace('.', '_')  # ví dụ 0_1%, 0_2%, ...
    poisoned_pool.to_csv(POISONED_POOL_DIR / f"ratio_{r_str}_percent.csv", index=False)
    base_pool.to_csv(BASE_POOL_DIR / f"ratio_{r_str}_percent.csv", index=False)

    print(f"✅ ratio={ratio*100:.1f}%: "
          f"{len(poisoned_pool)} poisoned_pool ({len(poisoned_part)} poison, {len(clean_part_for_poisoned)} clean) | "
          f"{len(base_pool)} base_pool ({len(targeted_part)} target, {len(clean_part_for_base)} clean)")

print("\n🎯 Hoàn tất tạo pool cho tất cả các ratio (0.1% → 3%)")

✅ ratio=0.1%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=0.5%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=1.0%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=1.5%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=2.0%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=2.5%: 5446 poisoned_pool (157 poison, 5289 clean) | 5446 base_pool (157 target, 5289 clean)
✅ ratio=3.0%: 5233 poisoned_pool (157 poison, 5076 clean) | 5233 base_pool (157 target, 5076 clean)

🎯 Hoàn tất tạo pool cho tất cả các ratio (0.1% → 3%)


In [7]:
import pandas as pd
from pathlib import Path
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
from tqdm import tqdm

DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
SPLIT_DIR = HOTFLIP_DIR / "split_docs"
EMB_DIR = HOTFLIP_DIR / "embeddings"
EMB_DIR.mkdir(parents=True, exist_ok=True)

# --- Thiết bị ---
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2", device=device)

# --- Sliding window config ---
MAX_TOKENS = 256
STRIDE = 128
BATCH_SIZE = 32

def sliding_window_embed(text, model, max_len=MAX_TOKENS, stride=STRIDE):
    """Cắt văn bản dài và trung bình embedding."""
    tokens = model.tokenizer.tokenize(text)
    if len(tokens) <= max_len:
        return model.encode([text], convert_to_tensor=True)
    else:
        chunks = []
        for i in range(0, len(tokens), stride):
            chunk = tokens[i:i + max_len]
            chunk_text = model.tokenizer.convert_tokens_to_string(chunk)
            chunks.append(chunk_text)
            if i + max_len >= len(tokens):
                break
        embeddings = model.encode(chunks, convert_to_tensor=True, batch_size=BATCH_SIZE)
        return embeddings.mean(dim=0, keepdim=True)

# --- Danh sách file và cột tương ứng ---
file_configs = {
    "poisoned_doc": "final_poisoned_doc",
    "targeted_doc": "document",
    "clean_doc": "document"
}

for name, text_col in file_configs.items():
    path = SPLIT_DIR / f"{name}.csv"
    df = pd.read_csv(path)

    print(f"\n🔹 Encoding {name} ({len(df)} docs) using column '{text_col}'...")
    all_embs = []
    for text in tqdm(df[text_col], desc=f"Encoding {name}"):
        emb = sliding_window_embed(str(text), model)
        all_embs.append(emb.cpu().numpy())

    all_embs = np.vstack(all_embs)
    np.save(EMB_DIR / f"{name}_emb.npy", all_embs)

    # chỉ giữ lại các cột liên quan để truy map theo id
    meta_cols = ["document_id", text_col]
    df[meta_cols].to_csv(EMB_DIR / f"{name}_meta.csv", index=False)

    print(f"✅ Done: {name} | shape = {all_embs.shape}")

print("\n🎯 Hoàn tất tính và lưu embedding cho 3 file gốc (clean / targeted / poisoned)")


🔹 Encoding poisoned_doc (157 docs) using column 'final_poisoned_doc'...


Encoding poisoned_doc: 100%|██████████| 157/157 [00:11<00:00, 13.40it/s]


✅ Done: poisoned_doc | shape = (157, 768)

🔹 Encoding targeted_doc (157 docs) using column 'document'...


Encoding targeted_doc: 100%|██████████| 157/157 [00:07<00:00, 21.95it/s]


✅ Done: targeted_doc | shape = (157, 768)

🔹 Encoding clean_doc (5289 docs) using column 'document'...


Encoding clean_doc: 100%|██████████| 5289/5289 [05:18<00:00, 16.60it/s]


✅ Done: clean_doc | shape = (5289, 768)

🎯 Hoàn tất tính và lưu embedding cho 3 file gốc (clean / targeted / poisoned)


In [9]:
# --- Encode queries (thêm vào sau khi đã encode 3 file gốc) ---
TEST_FILE = DATA_DIR / "test.csv"  # đường dẫn tới file queries

if TEST_FILE.exists():
    queries_df = pd.read_csv(TEST_FILE)
    # đảm bảo cột tên đúng
    assert "queries" in queries_df.columns or "query" in queries_df.columns, \
        "File test.csv cần có cột 'queries' (hoặc 'query')"

    # chọn tên cột đúng nếu khác
    q_col = "queries" if "queries" in queries_df.columns else "query"

    print(f"\n🔹 Encoding queries ({len(queries_df)} queries) using column '{q_col}'...")
    query_embs = []
    for text in tqdm(queries_df[q_col].fillna("").astype(str), desc="Encoding queries"):
        emb = sliding_window_embed(text, model)
        query_embs.append(emb.cpu().numpy())

    query_embs = np.vstack(query_embs)  # (n_queries, dim)
    np.save(EMB_DIR / "queries_emb.npy", query_embs)
    queries_df[["queries_id", q_col]] if "queries_id" in queries_df.columns else queries_df[[q_col]]
    # lưu metadata (giữ queries_id nếu có)
    meta_cols = ["queries_id", q_col] if "queries_id" in queries_df.columns else [q_col]
    queries_df[meta_cols].to_csv(EMB_DIR / "queries_meta.csv", index=False)

    print(f"✅ Done: queries embeddings saved with shape = {query_embs.shape}")
else:
    print(f"⚠️ Không tìm thấy file queries ở: {TEST_FILE}. Bỏ qua bước encode queries.")


🔹 Encoding queries (15232 queries) using column 'queries'...


Encoding queries: 100%|██████████| 15232/15232 [05:59<00:00, 42.39it/s]


✅ Done: queries embeddings saved with shape = (15232, 768)


In [17]:
import torch
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm

# ====================== CẤU HÌNH ======================

DATA_DIR = Path("/content/SANNER_2025/data")
HOTFLIP_DIR = DATA_DIR / "hotflip_result"
POISONED_POOL_DIR = HOTFLIP_DIR / "poisoned_pool"
EMB_DIR = HOTFLIP_DIR / "embeddings"
TEST_FILE = DATA_DIR / "test.csv"

# chọn thiết bị
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

TOP_K_LIST = [1, 3, 5, 10]

# ====================== HÀM CHÍNH ======================
def custom_precision_at_k_poison(query_embeds, pool_embs, pool_ids, poison_ids, K=10):
    sim = torch.matmul(query_embeds, pool_embs.T)
    topk_idx = torch.topk(sim, k=K, dim=1).indices

    total_hits = 0
    for i in range(sim.shape[0]):
        topk_docs = [pool_ids[idx] for idx in topk_idx[i].tolist()]
        total_hits += sum(d in poison_ids for d in topk_docs)

    N = query_embeds.shape[0]
    # ✅ scaled để kết quả nằm trong khoảng 1–10
    return (100 * total_hits) / (K * N)

# ====================== TẢI EMBEDDING ======================
print("🔹 Loading embeddings ...")

# queries
queries_emb = torch.tensor(np.load(EMB_DIR / "queries_emb.npy"), dtype=torch.float32, device=device)
queries_meta = pd.read_csv(EMB_DIR / "queries_meta.csv")

# poisoned / targeted / clean
meta_poisoned = pd.read_csv(EMB_DIR / "poisoned_doc_meta.csv")
meta_targeted = pd.read_csv(EMB_DIR / "targeted_doc_meta.csv")
meta_clean = pd.read_csv(EMB_DIR / "clean_doc_meta.csv")

emb_poisoned = torch.tensor(np.load(EMB_DIR / "poisoned_doc_emb.npy"), dtype=torch.float32, device=device)
emb_targeted = torch.tensor(np.load(EMB_DIR / "targeted_doc_emb.npy"), dtype=torch.float32, device=device)
emb_clean = torch.tensor(np.load(EMB_DIR / "clean_doc_emb.npy"), dtype=torch.float32, device=device)

# Map ID → embedding tensor
embedding_map = {
    **{doc_id: emb_poisoned[i] for i, doc_id in enumerate(meta_poisoned["document_id"])},
    **{doc_id: emb_targeted[i] for i, doc_id in enumerate(meta_targeted["document_id"])},
    **{doc_id: emb_clean[i] for i, doc_id in enumerate(meta_clean["document_id"])},
}

# ====================== CHẠY CHO TỪNG RATIO ======================
results = []

ratio_files = sorted(POISONED_POOL_DIR.glob("ratio_*percent.csv"))

for f in tqdm(ratio_files, desc="Evaluating poisoned_pool ratios"):
    ratio_name = f.stem
    df = pd.read_csv(f)

    # document_id → embedding
    valid_ids = [doc_id for doc_id in df["document_id"] if doc_id in embedding_map]
    pool_embs = torch.stack([embedding_map[doc_id] for doc_id in valid_ids]).to(device)
    pool_ids = list(valid_ids)

    # danh sách poisoned docs
    poison_ids = set(df.loc[df["chosen"] == 1, "document_id"].tolist())

    # tính cho từng K
    for K in TOP_K_LIST:
        p_at_k = custom_precision_at_k_poison(
            queries_emb, pool_embs, pool_ids, poison_ids, K=K
        )
        results.append({
            "ratio": ratio_name,
            "k": K,
            "custom_precision@k": round(float(p_at_k), 6),   # ✅ bỏ .item(), ép về float an toàn
            "num_queries": queries_emb.shape[0],
            "num_poisoned_docs": len(poison_ids),
            "num_pool_docs": len(pool_ids),
        })

# ====================== LƯU KẾT QUẢ ======================
results_df = pd.DataFrame(results)
out_path = HOTFLIP_DIR / "custom_p_at_k_poison.csv"
results_df.to_csv(out_path, index=False)

print(f"\n✅ Saved results to {out_path}")
print(results_df.head(10))

🔹 Loading embeddings ...


Evaluating poisoned_pool ratios: 100%|██████████| 7/7 [00:11<00:00,  1.67s/it]


✅ Saved results to /content/SANNER_2025/data/hotflip_result/custom_p_at_k_poison.csv
               ratio   k  custom_precision@k  num_queries  num_poisoned_docs  \
0  ratio_0_1_percent   1            3.571429        15232                157   
1  ratio_0_1_percent   3            3.437938        15232                157   
2  ratio_0_1_percent   5            3.453256        15232                157   
3  ratio_0_1_percent  10            3.409270        15232                157   
4  ratio_0_5_percent   1            3.571429        15232                157   
5  ratio_0_5_percent   3            3.437938        15232                157   
6  ratio_0_5_percent   5            3.453256        15232                157   
7  ratio_0_5_percent  10            3.409270        15232                157   
8  ratio_1_0_percent   1            3.571429        15232                157   
9  ratio_1_0_percent   3            3.437938        15232                157   

   num_pool_docs  
0           54